In [19]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

#### Link for the database -> https://thegymter.net/gymnast-database/
#### Link for individual athletes is https://thegymter.net/ {firs name}-{last name}/  

#### Example for Simone Biles: https://thegymter.net/simone-biles/

In [20]:
#URL = "https://thegymter.net/simone-biles/"  
#r = requests.get(URL)
#html_content = r.content

#soup = bs(html_content, 'html.parser')

# Showing sample HTTP request for Simone Biles
#print(soup.prettify())

In [21]:
#soup = bs(html_content, 'html.parser')

#tables = soup.findAll('table')   # All tables from the site. They are formatted the same.

# Indices are as follows:   0 -> Basic Info (Full name, nation, birthdate, active/inactive)
#                           1 -> Most recent competition year (2023 for most)
#                           2+ -> 2nd / nth most recent competition years

#number_comptetitions = (len(table)-1) # 9

#print(tables[0].prettify())

In [22]:
# Finding birthdate - Standard: 3rd column, 2nd row (indices 2 then 1)
#birth_date = tables[0].findAll('tr')[2].findAll('td')[1].text

#print(birth_date)

In [23]:
# Manualy checking the structure of tables

#for row in tables[1].find_all('tr'):
#    columns = row.find_all('td')
#    for column in columns: 
#        print(column.text)

In [24]:
#df_biles = pd.read_html(str(tables))[1]

In [25]:
#df_biles.head()

#### Getting list of athletes

In [26]:
url_gymter = "https://thegymter.net/gymnast-database/"

r = requests.get(url_gymter)
html_content = r.content

soup = bs(html_content, 'html.parser')

tr_tags = soup.findAll('tr')

links = []

for tag in tr_tags:
    a_tags= tag.find_all('a', href=True)
    for a_tag in a_tags:
        links.append(a_tag['href'])
        
print(links[1])   # Showing example link

https://thegymter.net/sihem-hamidi/


#### Creating the scraper

In [15]:
df_olympic = pd.DataFrame()

In [27]:
# updated scraper
def add_gymnast(url):
    global df_olympic 

    r = requests.get(str(url))
    html_content = r.content
    
    soup = bs(html_content, 'html.parser')
      
    tables = soup.findAll('table')
    
    full_name = tables[0].findAll('tr')[0].findAll('td')[1].text
    nation = tables[0].findAll('tr')[1].findAll('td')[1].text
    birth_date = tables[0].findAll('tr')[2].findAll('td')[1].text  
    
    # For gathering competition year: Listed in order
    strong_tags = soup.find_all('strong')
    comp_years = []
    
    for strong_tag in strong_tags: 
        header = strong_tag.get_text(strip=True)
        if 'Results' in header:
            year = header.split()[0]
            comp_years.append(year)
    
    
    for i in range(len(tables)-2):
        df_temp = pd.read_html(str(tables))[i+2]
        
        df_temp = df_temp.rename(columns=df_temp.iloc[0]).loc[1:]  # Making the first row the header and dropping that row
        
        df_temp['Full_Name'] = full_name
        df_temp['Nation'] = nation
        df_temp['Birthdate'] = birth_date
        
        df_temp['Comp_Year'] = comp_years[i]
        
        #df_olympic= df_olympic.append(df_temp, ignore_index=True)
        df_olympic = pd.concat([df_olympic, df_temp], ignore_index=True)



In [28]:
for link in links:
    try:
        add_gymnast(link)
    except: 
        print("Error scaping: " + link)

Error scaping: http://www.thegymter.net/dipa-karmakar
Error scaping: https://thegymter.net/malva-wingren/


In [29]:
df_olympic.head()

,Date,Competition,NaN,VT,UB,BB,FX,AA,Full_Name,Nation,Birthdate,comp_year,Comp_Year
0,Jul 23-Aug 8,Olympic Games,EF,——,——,14.000,——,——,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
1,Jul 23-Aug 8,Olympic Games,TF,13.766,——,——,——,——,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
2,Jul 23-Aug 8,Olympic Games,QF,14.966,14.566,14.066,14.133,57.731,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
3,Jun 24-27,U.S. Olympic Trials,AA,15.400,13.833,13.700,14.600,57.533,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
4,Jun 24-27,U.S. Olympic Trials,QF,15.466,14.600,15.133,15.366,60.565,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN


In [30]:
print(len(df_olympic))

23393


In [36]:
df_olympic = df_olympic.replace('——','')

In [37]:
df_olympic.head()

,Date,Competition,NaN,VT,UB,BB,FX,AA,Full_Name,Nation,Birthdate,comp_year,Comp_Year
0,Jul 23-Aug 8,Olympic Games,EF,,,14.000,,,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
1,Jul 23-Aug 8,Olympic Games,TF,13.766,,,,,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
2,Jul 23-Aug 8,Olympic Games,QF,14.966,14.566,14.066,14.133,57.731,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
3,Jun 24-27,U.S. Olympic Trials,AA,15.400,13.833,13.700,14.600,57.533,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN
4,Jun 24-27,U.S. Olympic Trials,QF,15.466,14.600,15.133,15.366,60.565,Simone Arianne Biles,United States,"March 14, 1997",2023,NaN


In [38]:
df_olympic.to_csv('Gym_Womens.csv', index=False)